In [1]:
## %gui osx
import random as rand
import numpy as np
import genelife as g

%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt

# import pygame as pg # already done in genelife
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))     # sets input cells to use full window width

import platform
print('System',platform.system())                                         # displays operating system
print('Platform',platform.platform())

def set_params(runp,simp):
    g.rulemod = g.runparams[0] = runp[0]
    g.repscheme = g.runparams[1] = runp[1]
    g.selection = g.runparams[2] = runp[2]
    g.overwritemask = g.runparams[3] = runp[3]
    g.survivalmask = g.runparams[4] = runp[4]
    g.colorfunction = g.runparams[5] = runp[5]
    g.initfield = g.runparams[6] = runp[6]
    g.birthmask = g.runparams[7] = runp[7]
    g.ancselectmask = g.runparams[6] = runp[6]
    g.colorfunction2 = g.runparams[9] = -1      # color function of 2nd window same as first
    g.nlog2pmut = g.simparams[0] = simp[0]
    g.initial1density = g.simparams[1] = simp[1]
    g.initialrdensity = g.simparams[2] = simp[2]
    g.ncoding = g.simparams[3] = simp[3]
    g.startgenechoice = g.simparams[4] = simp[4]
    g.ranseed = g.simparams[5] = simp[5]
    

from os import environ
environ['SDL_VIDEO_WINDOW_POS'] = "%d,%d" % (1000,32)   # set position of pygame (SDL) video window (x,y) pixel coords, values work well on iMac

Running with SDL version 2.0.8.


System Darwin
Platform Darwin-18.7.0-x86_64-i386-64bit


## Game of Life

In [2]:
# 2
#  selection=2 scissors-stone-well-paper
#  Note that sequences must differ for replication unless it is forced.
#  Prepared for recording and replay of gui user sequence
#  Color scheme: four different classes are red, green, blue and white
g.nrun=1; g.ndisp=1000; g.nskip=0; g.niter=100;  # simulation time stepping parameters
#nrun=1; ndisp=1; nskip=0; niter=1;         # single step
g.nhist = 10                                # set to n to turn on histogram configurations every nth step
g.nstat = 10                                # set to n to turn on statistics trace every nth step
g.rulemod = g.runparams[0] = 0              # 0,1 whether to allow GoL rule modifications
                                            # with rulemod 1 2-live-nb birth, 3-live-nb non-birth & non-survival possible
g.repscheme = g.runparams[1] = 0x0002       # bits 0 to 20 code for mostly independent binary options
g.selection = g.runparams[2] = 2            # fitness for 2 live neighbor rule : 0-6 see subgenelife.c code
g.overwritemask = g.runparams[3] = 3        # mask of 2 bits to overwrite instead of survival for 3(bit0) or 2(bit1) live nbs
g.survivalmask = g.runparams[4] = 0         # survive mask for two (bit 1) and three (bit 0) live neighbours : GoL = 3
g.colorfunction = g.runparams[5] = 0        # color function 0(hash), ≥1(fnal), 2 nongolchg yellow, 3 notgolrul yellow
g.initfield = g.runparams[6] = 0            # 1 init via 32x32 genepat.dat, n>1 init via nxn rand array
#g.birthmask = runparams[7] = 0x04          # for selection=8-13 this is the GoL birth mask
#g.ancselectmask = runparams[8] = 0xff      # bit mask for enabling gene-selective choice of ancestor for different birth rules
g.colorfunction2 = g.runparams[9] = -1      # color function of 2nd window same as first
g.nlog2pmut = g.simparams[0] = 0            # log2 gene mutation probability (0 or >56 means no mutation)
g.initial1density = g.simparams[1] = 8192   # initial 1 density in GOL state: 16384 = near 1/2 guaranteed C rand max val 32767=2^15-1
g.initialrdensity = g.simparams[2] = 0      # initial density of random genes
g.ncoding = g.simparams[3] = 0              # no of bits used to encode valid connection functions 1-16
g.startgenechoice = g.simparams[4] = 5      # initialize genes to startgene number 0-8 : 8 is random choice of 0-7
g.ranseed = g.simparams[5] = 1234           # random number generator initialization

g.genelife.initialize_planes(g.npoffsets)
g.genelife.initialize(g.runparams,g.simparams)
g.stepreg = []
g.pyreg = []
g.framenr = 0
g.cnt=0
g.rescale = True                            # if true, windows rescaled to 2x if possible (<= 512 in size), must restart kernel to change
g.show0(False)
# g.step()
# g.run(g.nrun, g.ndisp, g.nskip, g.niter, g.nhist, g.nstat, False,regfile='recording') # run this version to record, using right shift S to save
g.run(g.nrun, g.ndisp, g.nskip, g.niter, g.nhist, g.nstat, False) # run this version to replay

/Users/n/Projects/genelife/genelifepy/genelife.py:484: ExperimentalWarning: 'pixels2d' is in an experimental state.
  cgrid=sdl2.ext.pixels2d(surfacex1)
/Users/n/Projects/genelife/genelifepy/genelife.py:523: ExperimentalWarning: 'pixels2d' is in an experimental state.
  cgrid2=sdl2.ext.pixels2d(surface2x1)


## selection 10 near GoL

2 nearest 4nbr bits => doesn't die out.


In [9]:
# 10 GoL
#  selection=10 LUT with appropriate parameters for survival
#  this case uses the genes to encode the se (number of live edge-centred sites) and s dept rule tables for birth and survival
#  a fixed positional encoding is used in the gene with 1 bit per valid s,se combination
#  there are 1,2,3,4,5,4,3,2,1 configurations of se for s=0-8. Only 23 cases s=1-7 are lut allowed non-zero.
#  birth is determined by the and of the live neighbour genes (and 23 bits of birthmask) ie genecode
#  survival is determined the and of the live neighbour genes (and 23 bits of survivalmask) ie genecode
#  ncoding not used : only space for 1 bit per lut entry
#  this example starts with only allowing GoL configurations : extend by adding more bits to survival and birth masks
g.nrun=1; g.ndisp=1000; g.nskip=0; g.niter=1;# simulation time stepping parameters
g.nhist = 0                                  # set to n to turn on histogram configurations every nth step
g.nstat = 0                                  # set to n to turn on statistics trace every nth step
g.rulemod = g.runparams[0] = 0x1             # 0,1 whether to allow GoL rule modifications
g.repscheme = g.runparams[1] = 0x610         
g.selection     = g.runparams[2] = 10        # fitness for 2 live neighbor rule : 0-6 see subgenelife.c code
g.overwritemask = g.runparams[3] = 0x0      # mask of bits to overwrite if birth allowed and already occupied for bit position sum s
g.birthmask    = g.runparams[7] = 0x000001e0 # for selection=10-11 this is the GoL birth mask
g.birthmask    = g.runparams[7] = 0x000021e0 # GoL + 1 bit (nearest 4 nbr)
g.birthmask    = g.runparams[7] = 0x000031e4 # GoL + 3 bits (2 nearest 4 nbr + farthest 2nbr)
g.birthmask    = g.runparams[7] = 0x000031e0 # GoL + 2 bit (2 nearest 4 nbr)
g.birthmask    = g.runparams[7] = 0x000021e0 # GoL + 1 bit (nearest 4 nbr)


g.survivalmask = g.runparams[4] = 0x000001fc # for selection=10-11 this is the GoL survival mask
g.survivalmask = g.runparams[4] = 0x000031fc # GOL + 2 bits (2 nearest 4 nbr)
g.survivalmask = g.runparams[4] = 0x000021fc # GOL + 2 bits (2 nearest 4 nbr)
g.ancselectmask = g.runparams[8] = 0xff      # ancestor mask allowing selection for all s values
g.colorfunction = g.runparams[5] = 0         # color function can be cahnged with arrow keys
g.initfield = g.runparams[6] = 150           # 1 init via 32x32 genepat.dat, n>1 init via nxn rand array
g.nlog2pmut = g.simparams[0] = 8             # log2 gene mutation probability (0 or >56 means no mutation)
g.initial1density = g.simparams[1] = 8192    # initial 1 density in GOL state: 16384 = nearest to half of C rand max value 32767 = 2**15 - 1
g.initialrdensity = g.simparams[2] = 0       # initial density of random genes
g.ncoding = g.simparams[3] = 0               # for selection==8, lut, ncoding 1,2,3 bits per lut entry : 0 implies 3.
g.startgenechoice = g.simparams[4] = 8       # initialize genes to startgene number 0-8 : 8 is random choice of 0-7

g.genelife.initialize_planes(g.npoffsets)
g.genelife.initialize(g.runparams,g.simparams)
g.framenr = 0
g.cnt=0
g.show0()
#g.step(True)
g.run(g.nrun, g.ndisp, g.nskip, g.niter, g.nhist, g.nstat, False)

step 0 pixel data (0,245) gol 0 gene 0000000000000000 status 0000000000000000
step 57 pixel data (208,178) gol 0 gene 0000000000000000 status 0000000000000000
step 872 survivalmask changed to 11fc
step 927 birthmask changed to 11e0


## Daptics runs

### selection=9, glider_count optimization jper / density

In [12]:
g.nrun=1; g.ndisp=1000; g.nskip=0; g.niter=1;# simulation time stepping parameters
g.nhist = 0                                  # set to n to turn on histogram configurations every nth step
g.nstat = 0                                  # set to n to turn on statistics trace every nth stepg.runparams = np.array([  1, 493,   9, 121,  51,   0,   0,  30, 255,  -1], dtype='int32')
runp = np.array([  1, 486,   9, 249,  10,   0,   0,  86, 255,  -1], dtype='int32')
simp = np.array([     8,   8192,      0,      1,      8, 959893], dtype='int32')
set_params(runp,simp)
g.genelife.initialize_planes(g.npoffsets)
g.genelife.initialize(g.runparams,g.simparams)
g.framenr = 0
g.cnt=0
g.show0()
#g.step(True)
g.run(g.nrun, g.ndisp, g.nskip, g.niter, g.nhist, g.nstat, False)

step 38 pixel data (449,257) gol 0 gene 0000000000000000 status 0000000000000000
step 214 pixel data (224,226) gol 0 gene 0000000000000000 status 0000000000000000


### selection=9, glider_count optimization jper^2 / density^2

In [13]:
g.nrun=1; g.ndisp=1000; g.nskip=0; g.niter=1;# simulation time stepping parameters
g.nhist = 0                                  # set to n to turn on histogram configurations every nth step
g.nstat = 0                                  # set to n to turn on statistics trace every nth stepg.runparams = np.array([  1, 493,   9, 121,  51,   0,   0,  30, 255,  -1], dtype='int32')

runp = np.array([  1, 379,   9, 203, 152,   0,   0, 199, 255,  -1], dtype='int32')
simp = np.array([    8,  8192,     0,     1,     8, 67150], dtype='int32')
set_params(runp,simp)

g.genelife.initialize_planes(g.npoffsets)
g.genelife.initialize(g.runparams,g.simparams)
g.framenr = 0
g.cnt=0
g.show0()
#g.step(True)
g.run(g.nrun, g.ndisp, g.nskip, g.niter, g.nhist, g.nstat, False)

## Extra

### selection=9, glider_count optimization jper / density

In [9]:
g.nrun=1; g.ndisp=1000; g.nskip=0; g.niter=1;# simulation time stepping parameters
g.nhist = 0                                  # set to n to turn on histogram configurations every nth step
g.nstat = 0                                  # set to n to turn on statistics trace every nth stepg.runparams = np.array([  1, 493,   9, 121,  51,   0,   0,  30, 255,  -1], dtype='int32')
g.runparams = np.array([  1, 486,   9, 249,  10,   0,   0,  86, 255,  -1], dtype='int32')
g.simparams = np.array([     8,   8192,      0,      1,      8, 959893], dtype='int32')
g.genelife.initialize_planes(g.npoffsets)
g.genelife.initialize(g.runparams,g.simparams)
g.framenr = 0
g.cnt=0
g.show0()
#g.step(True)
g.run(g.nrun, g.ndisp, g.nskip, g.niter, g.nhist, g.nstat, False)